In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fundus_vessels_toolkit.seg2graph import RetinalVesselSeg2Graph, branches_by_nodes_to_node_graph
import cv2
import numpy as np
import os
import skimage.morphology as skmorph

In [3]:
from ipyprotopypes import ImageViewer
viewer = ImageViewer()

In [4]:
ID = 10
RAW_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/train/Original/1-images/"
VESSELS_PATH = "/home/gaby/These/Data/Fundus/Vessels/FIVES/train/Original/2-vessels/"
ID = os.listdir(RAW_PATH)[ID]
print(ID)
ID, suffix = ID.rsplit('.', 1)

raw = cv2.imread(RAW_PATH+ID+'.'+suffix)
vessels = cv2.imread(VESSELS_PATH+ID+'.png')

seg2graph = RetinalVesselSeg2Graph(raw.shape[0]//100)

print('raw: ', RAW_PATH+ID+'.jpg')
print('vessels: ', VESSELS_PATH+ID+'.png')

def to_float(img):
    return img.astype(float)/255

raw = to_float(raw)
vessels = to_float(vessels).mean(axis=2) > .5
skmorph.remove_small_holes(vessels, 5, 2, out=vessels)
skmorph.opening(vessels, np.ones((3,3)), out=vessels)
viewer.image = raw.transpose((2,0,1))


163_D.png
raw:  /home/gaby/These/Data/Fundus/Vessels/FIVES/train/Original/1-images/163_D.jpg
vessels:  /home/gaby/These/Data/Fundus/Vessels/FIVES/train/Original/2-vessels/163_D.png


In [5]:
def display(skel, skel_dist=None):
    bin_skel = skel>0
    raw_display = raw.copy()
    raw_display += np.expand_dims(vessels*.07, 2)
    if skel_dist is None:
        raw_display[bin_skel] = np.ones((3)) * .5
    else:
        raw_display[bin_skel] = np.expand_dims(skel_dist[bin_skel]/skel_dist.max(), 1)
    raw_display[skel==3] = [1,1,0]
    raw_display[skel==4] = [0,1,1]
    raw_display[skel==1] = [0,0,1]
    #raw_display[skel==2] = [1,1,1]
    #raw_display[skel==3] = [0,0,0]
    viewer.image = raw_display.transpose((2,0,1))
    
def display_label(label, jonctions=None, label_jonctions=True, label_branch=True):
    import colorsys
    import cv2
    
    bin_skel = skel>0
    raw_display = raw.copy()
    raw_display += np.expand_dims(vessels*.07, 2)
    
    bin_label = np.tile(np.expand_dims(label!=0, 2), (1,1,3))
    colors = np.asarray([colorsys.hsv_to_rgb(h, s, v) for h in np.linspace(0,1,28) for s in [.7, 1] for v in [.5, 1, .7]])
    colors = colors.reshape((7,-1,3)).transpose((1,0,2)).reshape((-1,3))
    for i, c in enumerate(colors):
        raw_display[(label!=0) & (label%len(colors)==i)] = c
    if label_branch:
        for i in range(1, label.max()+1):
            y, x = np.where(label==i)
            if len(y):
                y = int(np.mean(y)+10)
                x = int(np.mean(x)+10)
                color = colors[i%len(colors)]
                raw_display = cv2.putText(raw_display, str(i-1), (x, y), cv2.FONT_HERSHEY_PLAIN, .6, color)
            else:
                print(f'Branch {i-1} was not found')
            
    if jonctions is not None:
        jy, jx = np.round(jonctions).astype(np.int64)
        raw_display[jy, jx] = [1,1,1]
        if label_jonctions:
            for i, (y,x) in enumerate(zip(jy, jx)):
                raw_display = cv2.putText(raw_display, f":{i}", (x+3, y+3), cv2.FONT_HERSHEY_PLAIN, .6, (.6, .6, .6)) 
    viewer.image = raw_display.transpose((2,0,1))
viewer

ImageViewer()

In [12]:
skel = seg2graph.skeletonize(vessels)
display(skel)

In [15]:
seg2graph = RetinalVesselSeg2Graph(raw.shape[0]//80)
#seg2graph.max_spurs_distance = 0
# seg2graph.nodes_merge_distance = {'termination': 0, 'junction': 0, 'node': 0}
# seg2graph.merge_small_cycles = 0
seg2graph.simplify_topology = None
conn, branch_labels, node_yx = seg2graph.seg2adjacency(vessels, return_label=True)
display_label(branch_labels, node_yx)

In [9]:
print(f'Empty nodes: {[i for i, n in enumerate(conn.T) if n.sum()==0]}')
print(f'Irregular branches: {[i for i, b in enumerate(conn) if b.sum()!=2]}')
if len(node_yx[0]) != conn.shape[1]:
    print(f"Invalid number of nodes coordindates: {len(node_yx[0])} instead of {conn.shape[1]}.")
if np.max(branch_labels) != conn.shape[0]:
    print(f"Invalid number of branch labels: {np.max(branch_labels)} instead of {conn.shape[0]}.")

Empty nodes: []
Irregular branches: []


In [ ]:
graph = branches_by_nodes_to_node_graph(conn, node_yx)
edges = {}
branches = set()
for edge in graph.edges():
    e = graph.edges[edge]
    edges[f"Branch {e['branch']:03}"] = f"n{edge[0]}-{edge[1]}"
    branches |= {e['branch']}
    
for b in sorted(edges.keys()):
      print(b, edges[b])

In [ ]:
for i, b in enumerate(conn.T):
    print(f'Node {i} ({node_yx[0][i]:.0f}, {node_yx[1][i]:.0f}): ', np.where(b)[0])

In [ ]:
for i, b in enumerate(conn):
    print(f'Branch {i}: ', np.where(b)[0])

In [ ]:
conn.sum(axis=1).astype(int)

In [ ]:
np.sum(conn.sum(axis=1)>2)

In [ ]:
conn.sum(axis=1).shape